In [31]:
import os

In [35]:
snp_out_dir='/nrnb/users/btsui/Data/tcga_extracted_lgg_snp/'

In [38]:
processedFileS=pd.Series(os.listdir(snp_out_dir))

In [42]:
len(processedFileS)

1523

In [32]:
#''

manifest_dir='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/XGS_WGS/./tcga_lgg_wgs_bams.df.wxs_rnaseq.pickle'

import pandas as pd

n=pd.read_pickle(manifest_dir).shape[0]
print ('n of files for processing:',n)


n of files for processing: 1575


In [33]:
#!qstat

In [16]:
#!rm /cellar/users/btsui/Data/tcgaOut/*


In [17]:
os.system('qsub -t 1-{} ./align_tcga_snp_target.sh'.format(n))

0

In [27]:
#!qdel align_tcga_snp_target.sh

In [19]:
#!qstat -u btsui

depend on only the following script 

In [6]:
!cat align_tcga_snp_target.sh

#! /bin/bash
# Which shell to use
#$ -S /bin/bash
# Transfer all variables to job script (e.g. PATH, LD_LIBRARY_PATH, etc.)
#$ -V
# Queue to schedule jobs to
#$ -l h_rt=8:00:00
## -l u14
#$ -l h_vmem=1g
#$ -l long
#$ -l tmpfree=2T
#$ -l h=!(nrnb-5-0|nrnb-5-1|nrnb-5-2|nrnb-5-3|nrnb-5-4|nrnb-5-5|nrnb-5-6)
# Directory to send stdout and stderr
## -o /dev/null
## -e /dev/null
#$ -o /cellar/users/btsui/Data/tcgaOut/
#$ -e /cellar/users/btsui/Data/tcgaOut/
# Run in current working directory
#$ -cwd
# Array job 14175
#$ -t 1-100
## -p -0
#$ -r y
#$ -tc 20
#$ -pe smp 8
#SRR_FTP="/cellar/users/btsui/Data/SRA/META/processing.srr"
hostname
/cellar/users/btsui/anaconda3/bin/python ~/Project/METAMAP/notebook/RapMapTest/XGS_WGS/TestPipeline/extract_allele_all_targetted_ref.py $SGE_TASK_ID


In [7]:
manifest_dir='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/XGS_WGS/./tcga_lgg_wgs_bams.df.pickle'

In [9]:
pd.read_pickle(manifest_dir)['data_format'].value_counts()

BAM    1045
Name: data_format, dtype: int64

In [14]:
#!/cellar/users/btsui/anaconda3/bin/python ~/Project/METAMAP/notebook/RapMapTest/XGS_WGS/TestPipeline/extract_allele_all_targetted_ref.py 10

In [55]:
#!qstat -j align_tcga_snp_target.sh

In [51]:
#1.17G memory
!qalter align_tcga_snp_target.sh -pe smp 4 

modified parallel environment of job 408940
modified slot range of job 408940


In [54]:
!qalter align_tcga_snp_target.sh  -pe smp 2

modified parallel environment of job 408940
modified slot range of job 408940


In [12]:
#!qalter align_tcga_snp_target.sh  -pe smp 2

In [30]:
!ls -lath /nrnb/users/btsui/Data/tcga_extracted_lgg_snp/ | head

total 9.2G
-rw-r--r--  1 btsui users 5.2M Jul 19 08:58 37d2a30b-14b9-4846-81df-c254765bdacd.snp.txt.gz
drwxr-xr-x  2 btsui users 128K Jul 19 08:58 .
-rw-r--r--  1 btsui users 5.0M Jul 19 08:58 5aff639e-6963-4bc3-8891-c7299baf39a4.snp.txt.gz
-rw-r--r--  1 btsui users 5.5M Jul 19 08:58 7c095edb-8ba8-45cc-a320-d51906495bc9.snp.txt.gz
-rw-r--r--  1 btsui users 5.9M Jul 19 08:57 d7df37ac-d0d6-478a-93e7-e99136c3b357.snp.txt.gz
-rw-r--r--  1 btsui users 4.8M Jul 19 08:56 b9bc4c25-df56-426f-ad8a-4717ad301394.snp.txt.gz
-rw-r--r--  1 btsui users 5.4M Jul 19 08:49 c59ef5df-1ccd-4a99-a135-5df6edc5ca8b.snp.txt.gz
-rw-r--r--  1 btsui users 5.3M Jul 19 08:47 8ea2181b-3cfd-4d3b-a2d2-73fb4d3fc2f1.snp.txt.gz
-rw-r--r--  1 btsui users 4.3M Jul 19 08:47 ef84ab28-7064-4800-8312-8e517968daf9.snp.txt.gz
ls: write error: Broken pipe


In [34]:
!cat ./extract_allele_all_targetted_ref.py

import pandas as pd
import sys
import os
import time

#i=0
i=int(sys.argv[1])-1
#
manifest_dir='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/XGS_WGS/./tcga_lgg_wgs_bams.df.wxs_rnaseq.pickle'
token_dir='/cellar/users/ramarty/tokens/secret_token.visible.txt'
bam_read_count_dir='/cellar/users/btsui/Program/bam_read_count/bam-readcount-master/bin/bam-readcount'

gdc_cmd_fmt='gdc-client download -t {token_dir} -d {out_dir} {file_uuid}'
out_dir='/nrnb/users/btsui/Data/tcga_raw_lgg/'
snp_out_dir='/nrnb/users/btsui/Data/tcga_extracted_lgg_snp/'

manifest_df=pd.read_pickle(manifest_dir)
manifest_S=manifest_df.iloc[i]

file_uuid=manifest_S['file_id']
print ('UUID: ',file_uuid)

gdc_cmd=gdc_cmd_fmt.format(out_dir=out_dir,file_uuid=file_uuid,token_dir=token_dir)

result = os.system(gdc_cmd)
### pipe the data
tmpDir=out_dir+file_uuid+'/'
os.chdir(tmpDir)

os.system('rm fastq_pipe')

os.system('mkfifo fastq_pipe')
os.system('bamToFastq -i {} -fq fastq_pipe &'.format(manifest_S['file_name'

In [40]:
!ls

align_tcga.py			     submitJobsForTiming.ipynb
align_tcga_snp_target.sh	     tcga_extract_read_count.py
extract_allele_all_targetted_ref.py  test.sh
submitJobForAllleleExtraction.ipynb


In [65]:
#!df -h 

In [25]:
!ls -lath /nrnb/users/btsui/Data/tcga_extracted_lgg_snp/ | head

total 7.4G
-rw-r--r--  1 btsui users 3.7M Jul 18 17:48 1985b367-00c4-4c25-b049-5858e937cc6d.snp.txt.gz
drwxr-xr-x 15 btsui users 128K Jul 18 14:58 ..
drwxr-xr-x  2 btsui users 128K Jul 18 14:54 .
-rw-r--r--  1 btsui users   20 Jul 18 14:50 06c8a70b-3f0c-42bc-875a-f3e13c887a73.snp.txt.gz
-rw-r--r--  1 btsui users   20 Jul 18 14:50 b2bd014e-44fe-4d24-9b75-91baf36b3c0a.snp.txt.gz
-rw-r--r--  1 btsui users   20 Jul 18 14:50 eda26d7b-0725-46c4-9264-bcdf94dc163d.snp.txt.gz
-rw-r--r--  1 btsui users   20 Jul 18 14:50 37589627-0b38-41b0-b7a3-d2dd1f343550.snp.txt.gz
-rw-r--r--  1 btsui users   20 Jul 18 14:50 dcbaf670-8100-4c98-bd4d-2880a5805f23.snp.txt.gz
-rw-r--r--  1 btsui users   20 Jul 18 14:50 aeebf360-49c4-4db3-bdf4-daffdc5279cc.snp.txt.gz
ls: write error: Broken pipe
